# Conversational Voice AI with Knowledge Representation of Extended Warehouse Management

# Backend Technologies:  
   1. Azure -Cognitive AI      for NLP Services
   2. SAP S4 Embedded EWM 9.3   for Warehouse Operations.
   3. SAP ABAP                  for Server Side Validation.
   4. Python 3.3               for Client Side Validations.
   
### Integration Technologies:
   1. Azure Cognitive AI 
   2. Azure NLP (Text and Speech)
   3. SAP Restful API's. 
   4. WEB Hooks
                    
### Frontend Technologies:
   1. UI5 / Adaptive Cards/ Teams Channel
   2. Django / CSS / JS

In [4]:
# Import namespaces
from dotenv import load_dotenv
from datetime import datetime
import os
import azure.cognitiveservices.speech as speech_sdk
#Global Keys: Get keys for Azure Cognitive AI
GV_COGKEY = os.getenv('COG_SERVICE_KEY')
GV_COGREG = os.getenv('COG_SERVICE_REGION')

In [5]:
# Class to Get Speech Config Parameters by passing keys.
class GCL_S2T:
    # Define Method: Configure speech service
    # Pass the Global keys to Speech SDK and get Speech Config variables.
     
    def config_speech(self): 
        global speech_config
        speech_config = speech_sdk.SpeechConfig(GV_COGKEY, GV_COGREG)
        print('Ready to use speech service in:', speech_config.region)

    # Define Method Transcribe Command

    def TranscribeCommand(self):
        print(speech_config)
     # automatic language detection.
        auto_detect_source_language_config = speech_sdk.languageconfig.AutoDetectSourceLanguageConfig()
    # Configure speech recognition
        audio_config = speech_sdk.AudioConfig(use_default_microphone=True)
        speech_recognizer = speech_sdk.SpeechRecognizer(speech_config, audio_config)  
        print('Speak now...')

    # Process speech input
        speech = speech_recognizer.recognize_once_async().get()
        if speech.reason == speech_sdk.ResultReason.RecognizedSpeech:
             command = speech.text
             print(command)
        else:
            print(speech.reason)

    # Return the command
        return command

##Create Instance of the Speech Class
obj_s2t = GCL_S2T()
obj_s2t.config_speech() # Read Method config_speech
gv_question = obj_s2t.TranscribeCommand() # Transcribe Speech to Text


Ready to use speech service in: eastasia
Speak now...
Hello.


In [6]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.language.questionanswering import QuestionAnsweringClient
endpoint = "https://sanaai.cognitiveservices.azure.com/"
credential = AzureKeyCredential("d46d00379ac748419ab6c6c346699130")
knowledge_base_project = "SANA-QA"
deployment = "australiaeast"

class GCL_QA:
    def get_answer(self, gv_question):
        client = QuestionAnsweringClient(endpoint, credential)
        with client:
            question= gv_question
            output = client.get_answers(
            question = question,
            project_name=knowledge_base_project,
            deployment_name=deployment
            )
        return output
        
        
OBJ_QA = GCL_QA()
gv_output = OBJ_QA.get_answer(gv_question)

print("Q: {}".format(gv_question))
print("A: {}".format(gv_output.answers[0].answer))
gv_result = gv_output.answers[0].answer
print("Confidence Score: {}".format(gv_output.answers[0].confidence)) 
gv_confedence = int(gv_output.answers[0].confidence * 100)
print(gv_confedence)

/Users/I310202/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Q: Hello.
A: Hi!
Confidence Score: 1.0
100


In [ ]:
class T2S:
    def detection_to_speaker(self,text):    
    # create the auto detection language configuration without specific languages
        auto_detect_source_language_config = speech_sdk.languageconfig.AutoDetectSourceLanguageConfig()
        speech_synthesizer = speech_sdk.SpeechSynthesizer( speech_config=speech_config, auto_detect_source_language_config=auto_detect_source_language_config)
        result = speech_synthesizer.speak_text_async(text).get()        
        # Check result

        if result.reason == speech_sdk.ResultReason.SynthesizingAudioCompleted:
            print("Speech synthesized to speaker for text [{}]".format(text))
            stream = speech_sdk.AudioDataStream(result)
            stream.save_to_wav_file(r"output.wav")

read_text = gv_result
obj_t2s = T2S()
if gv_confedence > 30:
    lv_text = "with an accuray of %d" %gv_confedence

    #lv_text = "with an accuracy of " + str(gv_confedence) + " Percentage, here is my answer"
    obj_t2s.detection_to_speaker(lv_text)
    obj_t2s.detection_to_speaker(read_text)
else:
    obj_t2s.detection_to_speaker("I am not so Sure about it! ")


NameError: name 'gv_result' is not defined